In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re  # 정규 표현식 모듈

# 예시 CSV 파일 경로 (실제 CSV 파일 경로를 입력하세요)
file_path = '/content/drive/MyDrive/업종별 기업경기실사지수, 2009.8~(실적)_14204004.csv'

# CSV 파일에서 데이터를 읽어오기
df = pd.read_csv(file_path)

# 데이터의 첫 5개 행을 출력하여 날짜 열 이름 확인
print(df.head())

# 날짜 열이 '2009-08-01' 형태로 되어 있다면, 이를 'Aug-09'와 같은 형식으로 변환
# 먼저 데이터의 열 이름을 확인해봅니다. 예를 들어 'Aug-09'와 같은 열이 없다면, '2009-08-01' 형식의 열이 있을 것입니다.

# 열 이름에서 '업황실적BSI 1)'와 같은 불필요한 기호나 공백 제거
df.columns = df.columns.str.replace(r'\s*\d+\)$', '', regex=True)  # 숫자와 ')' 제거
df.columns = df.columns.str.strip()  # 앞뒤 공백 제거

# 이제 열 이름을 확인하여 '업황실적BSI', '매출실적BSI'처럼 필요한 항목만 남은 상태인지 확인
print(df.columns)

# 날짜 열 목록 추출 (열 이름에 '-'이 포함된 열만 추출)
date_columns = [col for col in df.columns if '-' in col]

# 날짜 형식 변환: CSV에서 날짜를 'yyyy-mm-dd'로 읽었기 때문에 이를 'Aug-09' 형태로 변환
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.strftime('%b-%y')  # 'Aug-09' 형태로 변환


print(df[date_columns].head())

# BSI 항목의 우선순위 및 중요도 설정
importance_weights = {
    '업황실적BSI': 0.15,  # 예: 업황실적은 전반적인 경기 민감도가 크므로 중요
    '매출실적BSI': 0.12,  # 매출 실적은 기업의 기본적인 성과로 중요
    '수출실적BSI': 0.10,  # 수출 실적은 글로벌 경제 변화에 영향을 받음
    '내수판매실적BSI': 0.10,  # 내수도 중요하지만 상대적으로 낮은 비중
    '생산실적BSI': 0.10,  # 생산실적은 생산성이 반영되므로 중요
    '신규수주실적BSI': 0.12,  # 신규 수주는 미래 성장을 결정짓는 중요한 지표
    '제품재고실적BSI': 0.08,  # 재고 수준은 중요하지만 상대적으로 낮은 비중
    '가동률실적BSI': 0.08,  # 가동률은 생산 효율성을 나타내므로 중요
    '생산설비실적BSI': 0.06,  # 설비 투자는 중요하지만 상대적으로 낮은 비중
    '설비투자실적BSI': 0.06,  # 설비 투자도 중요하지만 상대적으로 낮은 비중
    '채산성실적BSI': 0.06,  # 채산성은 수익성에 영향을 미치므로 중요한 항목
    '원자재구입가격실적BSI': 0.04,  # 원자재 가격은 중요하지만 상대적 중요도 낮음
    '제품판매가격실적BSI': 0.04,  # 판매가격은 가격 전쟁에서 중요한 요소지만 낮은 비중
    '자금사정실적BSI': 0.08,  # 자금사정은 부도에 큰 영향을 미침
    '인력사정실적BSI': 0.04   # 인력 사정은 상대적으로 적은 비중
}

# 업종별 가중치 설정 (업종별로 다른 가중치를 적용)
# 이 부분은 업종별로 다르게 가중치를 설정합니다.
# 예시로 각 업종에 대해 임의의 가중치를 설정 (실제로는 더 정교한 분석을 통해 설정)
sector_weights = {
    '전산업': 0.1,
    '제조업': 0.15,
    '식료품': 0.05,
    '음료': 0.05,
    '섬유': 0.05,
    '의복모피': 0.05,
    '가죽·가방·신발': 0.05,
    '목재·나무': 0.05,
    '펄프·종이': 0.05,
    '화학물질·제품': 0.1,
    '자동차': 0.1,
    # 나머지 업종에 대한 가중치는 추가로 설정 가능
}

# 업종별 가중치를 데이터프레임에 매핑
df['가중치'] = df['업종코드'].map(sector_weights)

# 각 BSI 항목에 대해 중요도에 맞는 가중치를 적용하여 가중 평균을 계산합니다.
def weighted_average(row):
    weighted_values = 0
    for col in date_columns:
        # 해당 BSI 항목의 중요도를 가져와서 가중치를 적용
        bsi_name = col.split('-')[0]  # 예: '업황실적BSI'
        weight = importance_weights.get(bsi_name, 0)  # 기본 가중치는 0
        weighted_values += row[col] * weight
    return weighted_values

# 연도별 종합 변수 계산
df['종합 변수'] = df.apply(weighted_average, axis=1)

# 결과 출력 (업종별로 종합 변수 확인)
print(df[['업종코드', '종합 변수']])

# CSV 파일로 저장 (결과를 새로운 CSV 파일로 저장)
df.to_csv('업종별_종합_결과.csv', index=False)

# 종합 지표를 연도별로 평균 계산하여 CSV 파일로 저장할 수도 있음
df_melted = pd.melt(df, id_vars=['업종코드', '가중치'], value_vars=date_columns, var_name='년도', value_name='BSI')

# 연도별 평균 종합 지수 계산
annual_summary = df_melted.groupby('년도')['BSI'].mean().reset_index()

# 연도별 평균 종합 지표 CSV로 저장
annual_summary.to_csv('연도별_종합_지표.csv', index=False)


                                  통계표       BSI코드     업종코드  단위   변환  2009/08  \
0  6.1.2.1. 업종별 기업경기실사지수, 2009.8~(실적)  업황실적BSI 1)      전산업 NaN  원자료     81.0   
1  6.1.2.1. 업종별 기업경기실사지수, 2009.8~(실적)  업황실적BSI 1)      제조업 NaN  원자료     84.0   
2  6.1.2.1. 업종별 기업경기실사지수, 2009.8~(실적)  업황실적BSI 1)      식료품 NaN  원자료     99.0   
3  6.1.2.1. 업종별 기업경기실사지수, 2009.8~(실적)  업황실적BSI 1)       음료 NaN  원자료    103.0   
4  6.1.2.1. 업종별 기업경기실사지수, 2009.8~(실적)  업황실적BSI 1)       섬유 NaN  원자료     72.0   

   2009/09  2009/10  2009/11  2009/12  ...  2023/03  2023/04  2023/05  \
0     84.0     88.0     85.0     89.0  ...     72.0     72.0     76.0   
1     90.0     90.0     88.0     89.0  ...     70.0     70.0     73.0   
2    104.0     94.0     94.0     96.0  ...     76.0     81.0     80.0   
3     96.0     91.0     80.0     90.0  ...     72.0     80.0     73.0   
4     76.0     76.0     62.0     79.0  ...     46.0     58.0     47.0   

   2023/06  2023/07  2023/08  2023/09  2023/10  2023/11  2023/12  
0     76.0   

In [ ]:
import pandas as pd
import re  # 정규 표현식 모듈

# 예시 CSV 파일 경로 (실제 CSV 파일 경로를 입력하세요)
file_path = '/content/drive/MyDrive/업종별 기업경기실사지수, 2009.8~(실적)_14214156.csv'

# CSV 파일에서 데이터를 읽어오기
df = pd.read_csv(file_path,  encoding='utf-8')

# 데이터의 첫 5개 행을 출력하여 날짜 열 이름 확인
df = df.drop(['통계표', '단위', '변환'], axis=1)

df.fillna(0, inplace=True)

# BSI코드 열에서 항목명만 추출 (숫자와 괄호 제거)
def extract_bsi_name(bsi_code):
    # 정규식: 숫자와 괄호를 제거하고, 항목명만 추출
    match = re.match(r'([^\d\(\)]+)', bsi_code)
    return match.group(1).strip() if match else bsi_code.strip()

# BSI코드 열에서 항목명만 추출하여 새로운 열로 저장
df['BSI항목'] = df['BSI코드'].apply(extract_bsi_name)

# 추출된 BSI항목 출력 (정상적으로 항목명이 추출되는지 확인)
#print(df[['BSI코드', 'BSI항목']].head())

# BSI 항목의 우선순위 및 중요도 설정
importance_weights = {
    '업황실적BSI': 0.15,
    '매출실적BSI': 0.12,
    '수출실적BSI': 0.10,
    '내수판매실적BSI': 0.10,
    '생산실적BSI': 0.10,
    '신규수주실적BSI': 0.12,
    '제품재고실적BSI': 0.08,
    '가동률실적BSI': 0.08,
    '생산설비실적BSI': 0.06,
    '설비투자실적BSI': 0.06,
    '채산성실적BSI': 0.06,
    '원자재구입가격실적BSI': 0.04,
    '제품판매가격실적BSI': 0.04,
    '자금사정실적BSI': 0.08,
    '인력사정실적BSI': 0.04
}


# 날짜 열 목록 추출 (열 이름에 '-'이 포함된 열만 추출)
date_columns = [col for col in df.columns if re.match(r'\d{4}/\d{2}', col)]


def weighted_average(row, year):
    weighted_values = 0

    for col in date_columns:
        bsi_name = row['BSI항목']
        weight = importance_weights.get(bsi_name, 0)  # 기본 가중치는 0
        weighted_values += float(row[col]) * weight
    return weighted_values


# 연도별 종합 변수 계산
df['종합 변수'] = df.apply(weighted_average, axis=1)

# 결과 출력 (업종별로 종합 변수 확인)
#print(df[['업종코드', '종합 변수']])

# CSV 파일로 저장 (결과를 새로운 CSV 파일로 저장)
df.to_csv('업종별_종합_결과.csv', index=False)

# 종합 지표를 연도별로 평균 계산하여 CSV 파일로 저장할 수도 있음
df_melted = pd.melt(df, id_vars=['업종코드', '가중치'], value_vars=date_columns, var_name='년도', value_name='BSI')

# 연도별 평균 종합 지수 계산
annual_summary = df_melted.groupby('년도')['BSI'].mean().reset_index()

# 연도별 평균 종합 지표 CSV로 저장
annual_summary.to_csv('연도별_종합_지표.csv', index=False)


In [ ]:
import pandas as pd

# 1. CSV 파일을 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_15214328.csv')

# 2. 필요한 열만 추출
# 열 목록: '통계표', 'CSI코드', 'CSI분류코드', '단위', '2008/07', '2008/08', ...


columns_to_keep = ['CSI코드', 'CSI분류코드']

colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2008, 2024) for month in range(1, 13)]

colums_to_number = [col for col in colums_to_number if col not in colums_to_number]

colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number

df = df[colums_to_total]

df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. 각 CSI 항목을 큰 카테고리로 분류 (예시로 일부 카테고리 분류)
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.6, '현재경기판단CSI': 0.4}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {'생활형편전망CSI': 0.25, '향후경기전망CSI': 0.25, '취업기회전망CSI': 0.25, '금리수준전망CSI': 0.25}
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI', '내구재지출전망CSI', '의류비지출전망CSI',
                 '외식비지출전망CSI', '여행비지출전망CSI', '교육비지출전망CSI', '의료보건비지출전망CSI',
                 '교양오락문화생활비지출전망CSI', '교통비및통신비지출전망CSI', '주거비지출전망CSI'],
        '항목별가중치': {
            '가계수입전망CSI': 0.1, '소비지출전망CSI': 0.1, '내구재지출전망CSI': 0.1, '의류비지출전망CSI': 0.05,
            '외식비지출전망CSI': 0.1, '여행비지출전망CSI': 0.05, '교육비지출전망CSI': 0.1, '의료보건비지출전망CSI': 0.1,
            '교양오락문화생활비지출전망CSI': 0.1, '교통비및통신비지출전망CSI': 0.05, '주거비지출전망CSI': 0.1
        }
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI', '가계부채전망CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.25, '가계저축전망CSI': 0.25, '현재가계부채CSI': 0.25, '가계부채전망CSI': 0.25}
    },

    '물가전망조사': {
        '항목': ['물가수준전망(1년후)CSI', '주택가격전망CSI', '임금수준전망CSI', '물가수준전망(6개월후)CSI(~2012.12월)'],
        '항목별가중치': {'물가수준전망(1년후)CSI': 0.25, '주택가격전망CSI': 0.25, '임금수준전망CSI': 0.25, '물가수준전망(6개월후)CSI(~2012.12월)': 0.25}
    },
    '자산가치전망조사': {
        '항목': ['주택상가가치전망CSI', '토지임야가치전망CSI', '금융저축가치전망CSI', '주식가치전망CSI'],
        '항목별가중치': {'주택상가가치전망CSI': 0.25, '토지임야가치전망CSI': 0.25, '금융저축가치전망CSI': 0.25, '주식가치전망CSI': 0.25}
    }
}

# 4. 사회경제적 특성별 가중치 설정 (예시로 설정한 값들)
demographic_weights = {
    '성별': {'남자': 1.2, '여자': 1.0},
    '연령대': {'40세미만': 1.0, '40-50세': 1.1, '50-60세': 1.2, '60-70세': 1.3, '70세이상': 1.4},
    '직업': {'봉급생활자': 1.0, '자영업자': 1.1, '기타': 1.2},
    '소득수준': {'100만원미만': 1.2, '100-200만원': 1.0, '200-300만원': 1.1, '300-400만원': 1.2,
                '400-500만원': 1.3, '500만원이상': 1.4},
    '주거상황': {'자가': 1.0, '임차등': 1.1},
    '거주지역': {'서울': 1.2, '6대광역시': 1.1, '기타도시': 1.0},
}

# 5. CSI 데이터가 주어진 형태에 맞게 변환
# 데이터가 'CSI분류코드'와 '통계표'에 따라 CSI 값을 가지므로, 이를 피벗하여 다룬다.


df_csi = df.pivot_table(index=['CSI코드', 'CSI분류코드'], values=colums_to_number, aggfunc='first')

df.fillna(0, inplace=True)

#print(df_csi["2008/07"]["현재생활형편CSI"]["100-200만원"])


def calculate_weighted_score(df_csi, category, year):
    weighted_scores = []
    weighted_sum = 0

    # 각 사회경제적 특성에 대한 가중치 적용
    # 각 CSI 항목에 대한 가중치 적용
    for demographic_subgroup, factor in demographic_factor.items():
        for item, weight in categories[category]['항목별가중치'].items():
                weighted_sum  += df_csi[year][item][demographic_subgroup] * weight

        weighted_sum = weighted_sum * factor

    weighted_scores.append(weighted_sum)
    weighted_sum = 0

    # 해당 카테고리 내 모든 항목들의 가중 평균을 계산

    return sum(weighted_scores)/len(weighted_scores)

# 8. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi, demographic_weights):
    combined_index = {}
    years = colums_to_number  # 2008~2023년

    # 8-1. 사회경제적 특성들의 모든 조합을 고려하여 가중치 계산
    for category in categories:
        category_score = {}

        #년도별로 종합지표 정리
        for year in years:
           category_score[year] = calculate_weighted_score(df_csi, category, year)

        combined_index[category] = category_score

    return combined_index

# 9. 결과 출력 및 CSV 파일로 저장

combined_index = compute_combined_index_for_all_years(df_csi, demographic_weights)
print(combined_index)

# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index)

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")


<ipython-input-4-3e862766d22d>:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
<ipython-input-4-3e862766d22d>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


{'경제인식조사': {'2008/07': 306.372496, '2008/08': 372.64524800000004, '2008/09': 358.82770666666664, '2008/10': 315.093904, '2008/11': 317.145232, '2008/12': 300.9463253333333, '2009/01': 327.039536, '2009/02': 347.2638346666667, '2009/03': 325.67288, '2009/04': 434.52807466666667, '2009/05': 498.8281333333334, '2009/06': 515.4819146666667, '2009/07': 532.0232533333334, '2009/08': 584.2597706666667, '2009/09': 588.4402186666667, '2009/10': 612.943072, '2009/11': 583.759856, '2009/12': 569.928112, '2010/01': 567.7548479999999, '2010/02': 554.2262666666667, '2010/03': 553.9489706666667, '2010/04': 551.416096, '2010/05': 558.3872160000001, '2010/06': 570.585856, '2010/07': 569.844384, '2010/08': 567.466192, '2010/09': 537.857888, '2010/10': 528.068544, '2010/11': 549.0455306666667, '2010/12': 539.9358026666667, '2011/01': 522.7449813333334, '2011/02': 505.04874133333334, '2011/03': 438.79718399999996, '2011/04': 457.77635200000003, '2011/05': 492.7746400000001, '2011/06': 482.35787733333336, 

In [5]:
import pandas as pd

# 1. CSV 파일을 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_23011326.csv')


# 2. 필요한 열만 추출
columns_to_keep = ['CSI코드', 'CSI분류코드']  # 필수 컬럼
colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2009, 2023) for month in range(1, 13)]



# 제외할 열을 반영하여 컬럼 목록을 정리
colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number

df = df[colums_to_total]

# 쉼표 및 공백 제거
df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. 각 CSI 항목을 큰 카테고리로 분류
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.6, '현재경기판단CSI': 0.4}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {'생활형편전망CSI': 0.25, '향후경기전망CSI': 0.25, '취업기회전망CSI': 0.25, '금리수준전망CSI': 0.25}
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI', '내구재지출전망CSI', '의류비지출전망CSI',
                 '외식비지출전망CSI', '여행비지출전망CSI', '교육비지출전망CSI', '의료보건비지출전망CSI',
                 '교양오락문화생활비지출전망CSI', '교통비및통신비지출전망CSI'],
        '항목별가중치': {
            '가계수입전망CSI': 0.1, '소비지출전망CSI': 0.1, '내구재지출전망CSI': 0.1, '의류비지출전망CSI': 0.05,
            '외식비지출전망CSI': 0.1, '여행비지출전망CSI': 0.05, '교육비지출전망CSI': 0.1, '의료보건비지출전망CSI': 0.1,
            '교양오락문화생활비지출전망CSI': 0.1, '교통비및통신비지출전망CSI': 0.05, '주거비지출전망CSI': 0.1
        }
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI', '가계부채전망CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.25, '가계저축전망CSI': 0.25, '현재가계부채CSI': 0.25, '가계부채전망CSI': 0.25}
    }
}

# 4. CSI 데이터를 피벗하여 다룬다.
df_csi = df.pivot_table(index=['CSI코드', 'CSI분류코드'], values=colums_to_number, aggfunc='first')

# NaN 값을 0으로 대체
df.fillna(0, inplace=True)

# 5. 가중치를 적용하여 각 카테고리별 종합 점수 계산
def calculate_weighted_score(df_csi, category, year):
    weighted_sum = 0

    # 각 CSI 항목에 대한 가중치 적용
    for item, weight in categories[category]['항목별가중치'].items():
        weighted_sum += df_csi.loc[item, year] * weight

    return weighted_sum

# 6. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi):
    combined_index = {}
    years = [str(year) for year in range(2009, 2023)]  # 2008~2023년

    # 각 카테고리별로 종합 점수 계산
    for category in categories:
        category_score = {}

        # 연도별로 종합지표 계산
        for year in years:
            category_score[year] = calculate_weighted_score(df_csi, category, year)

        combined_index[category] = category_score

    return combined_index

# 7. 결과 출력 및 CSV 파일로 저장
combined_index = compute_combined_index_for_all_years(df_csi)
print(combined_index)

# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index)

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")


<ipython-input-5-77f2d9b24e20>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
<ipython-input-5-77f2d9b24e20>:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


KeyError: '2009'

In [6]:
import pandas as pd

# 1. CSV 파일을 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_15214328.csv')

# 2. 필요한 열만 추출
columns_to_keep = ['CSI코드', 'CSI분류코드']  # 필수 컬럼
colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2009, 2024) for month in range(1, 13)]

# 제외할 열을 반영하여 컬럼 목록을 정리
colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number

df = df[colums_to_total]

# 쉼표 및 공백 제거
df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. 각 CSI 항목을 큰 카테고리로 분류
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.6, '현재경기판단CSI': 0.4}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {'생활형편전망CSI': 0.25, '향후경기전망CSI': 0.25, '취업기회전망CSI': 0.25, '금리수준전망CSI': 0.25}
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI', '내구재지출전망CSI', '의류비지출전망CSI',
                 '외식비지출전망CSI', '여행비지출전망CSI', '교육비지출전망CSI', '의료보건비지출전망CSI',
                 '교양오락문화생활비지출전망CSI', '교통비및통신비지출전망CSI', '주거비지출전망CSI'],
        '항목별가중치': {
            '가계수입전망CSI': 0.1, '소비지출전망CSI': 0.1, '내구재지출전망CSI': 0.1, '의류비지출전망CSI': 0.05,
            '외식비지출전망CSI': 0.1, '여행비지출전망CSI': 0.05, '교육비지출전망CSI': 0.1, '의료보건비지출전망CSI': 0.1,
            '교양오락문화생활비지출전망CSI': 0.1, '교통비및통신비지출전망CSI': 0.05, '주거비지출전망CSI': 0.1
        }
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI', '가계부채전망CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.25, '가계저축전망CSI': 0.25, '현재가계부채CSI': 0.25, '가계부채전망CSI': 0.25}
    },
    '물가전망조사': {
        '항목': ['물가수준전망(1년후)CSI', '주택가격전망CSI', '임금수준전망CSI', '물가수준전망(6개월후)CSI(~2012.12월)'],
        '항목별가중치': {'물가수준전망(1년후)CSI': 0.25, '주택가격전망CSI': 0.25, '임금수준전망CSI': 0.25, '물가수준전망(6개월후)CSI(~2012.12월)': 0.25}
    },
    '자산가치전망조사': {
        '항목': ['주택상가가치전망CSI', '토지임야가치전망CSI', '금융저축가치전망CSI', '주식가치전망CSI'],
        '항목별가중치': {'주택상가가치전망CSI': 0.25, '토지임야가치전망CSI': 0.25, '금융저축가치전망CSI': 0.25, '주식가치전망CSI': 0.25}
    }
}

# 4. CSI 데이터를 피벗하여 다룬다.
df_csi = df.pivot_table(index=['CSI코드', 'CSI분류코드'], values=colums_to_number, aggfunc='first')

# NaN 값을 0으로 대체
df.fillna(0, inplace=True)

# 5. 연도별로 종합 지표 계산 (연도별로 계산해야 하므로, '년도/월' 데이터를 연도별로 합산하거나 평균화)
def calculate_weighted_score(df_csi, category, year):
    weighted_sum = 0

    # 각 CSI 항목에 대한 가중치 적용
    for item, weight in categories[category]['항목별가중치'].items():
        # 연도별 평균 계산
        monthly_columns = [col for col in df_csi.columns if col.startswith(year)]  # 연도별 월별 데이터를 선택
        monthly_values = df_csi.loc[item, monthly_columns]
        weighted_sum += monthly_values.mean() * weight  # 월별 값의 평균을 구해 가중치를 곱함

    return weighted_sum

# 6. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi):
    combined_index = {}
    years = [str(year) for year in range(2008, 2024)]  # 2008~2023년

    # 각 카테고리별로 종합 점수 계산
    for category in categories:
        category_score = {}

        # 연도별로 종합지표 계산
        for year in years:
            category_score[year] = calculate_weighted_score(df_csi, category, year)

        combined_index[category] = category_score

    return combined_index

# 7. 결과 출력 및 CSV 파일로 저장
combined_index = compute_combined_index_for_all_years(df_csi)
print(combined_index)

# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index)

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")


<ipython-input-6-441ff4d40f26>:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
<ipython-input-6-441ff4d40f26>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


{'경제인식조사': {'2008': Series([], dtype: float64), '2009': 2009/01     56.363636
2009/02     60.027273
2009/03     56.218182
2009/04     74.981818
2009/05     86.090909
2009/06     88.900000
2009/07     91.963636
2009/08    100.790909
2009/09    101.509091
2009/10    105.681818
2009/11    100.636364
2009/12     98.190909
dtype: float64, '2010': 2010/01    97.990909
2010/02    95.609091
2010/03    95.509091
2010/04    95.236364
2010/05    96.363636
2010/06    98.309091
2010/07    98.109091
2010/08    97.790909
2010/09    92.872727
2010/10    91.245455
2010/11    94.672727
2010/12    93.236364
dtype: float64, '2011': 2011/01    90.218182
2011/02    87.418182
2011/03    76.036364
2011/04    79.309091
2011/05    85.181818
2011/06    83.227273
2011/07    82.136364
2011/08    78.445455
2011/09    75.172727
2011/10    76.936364
2011/11    82.036364
2011/12    78.000000
dtype: float64, '2012': 2012/01    75.709091
2012/02    78.672727
2012/03    80.245455
2012/04    83.281818
2012/05    86.045455

In [7]:
import pandas as pd

# 1. CSV 파일을 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_15214328.csv')

# 제외할 열 목록 (2008년 1월~6월)
exclude_columns = ['2008/01', '2008/02', '2008/03', '2008/04', '2008/05', '2008/06']

# 2. 필요한 열만 추출
columns_to_keep = ['CSI코드', 'CSI분류코드']  # 필수 컬럼
colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2008, 2024) for month in range(1, 13)]

# 제외할 열을 반영하여 컬럼 목록을 정리
colums_to_number = [col for col in colums_to_number if col not in exclude_columns]
colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number

df = df[colums_to_total]

# 쉼표 및 공백 제거
df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. 제외할 항목들 (물가수준전망(1년후)CSI 등) 목록
exclude_items = [
    '물가수준전망(1년후)CSI', '주택가격전망CSI', '임금수준전망CSI',
    '물가수준전망(6개월후)CSI(~2012.12월)', '주택·상가가치전망CSI',
    '토지·임야가치전망CSI', '금융저축가치전망CSI'
]

# 4. 각 CSI 항목을 큰 카테고리로 분류
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.6, '현재경기판단CSI': 0.4}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {'생활형편전망CSI': 0.25, '향후경기전망CSI': 0.25, '취업기회전망CSI': 0.25, '금리수준전망CSI': 0.25}
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI', '내구재지출전망CSI', '의류비지출전망CSI',
                 '외식비지출전망CSI', '여행비지출전망CSI', '교육비지출전망CSI', '의료보건비지출전망CSI',
                 '교양오락문화생활비지출전망CSI', '교통비및통신비지출전망CSI', '주거비지출전망CSI'],
        '항목별가중치': {
            '가계수입전망CSI': 0.1, '소비지출전망CSI': 0.1, '내구재지출전망CSI': 0.1, '의류비지출전망CSI': 0.05,
            '외식비지출전망CSI': 0.1, '여행비지출전망CSI': 0.05, '교육비지출전망CSI': 0.1, '의료보건비지출전망CSI': 0.1,
            '교양오락문화생활비지출전망CSI': 0.1, '교통비및통신비지출전망CSI': 0.05, '주거비지출전망CSI': 0.1
        }
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI', '가계부채전망CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.25, '가계저축전망CSI': 0.25, '현재가계부채CSI': 0.25, '가계부채전망CSI': 0.25}
    }
}

# 5. CSI 데이터를 피벗하여 다룬다.
df_csi = df.pivot_table(index=['CSI코드', 'CSI분류코드'], values=colums_to_number, aggfunc='first')

# NaN 값을 0으로 대체
df.fillna(0, inplace=True)

# 6. 연도별로 종합 지표 계산 (연도별로 계산해야 하므로, '년도/월' 데이터를 연도별로 합산하거나 평균화)
def calculate_weighted_score(df_csi, category, year):
    weighted_sum = 0

    # 각 CSI 항목에 대한 가중치 적용
    for item, weight in categories[category]['항목별가중치'].items():
        # 제외된 항목을 확인하고 건너뛰기
        if item in exclude_items:
            continue

        # 연도별 평균 계산
        monthly_columns = [col for col in df_csi.columns if col.startswith(year)]  # 연도별 월별 데이터를 선택
        monthly_values = df_csi.loc[item, monthly_columns]
        weighted_sum += monthly_values.mean() * weight  # 월별 값의 평균을 구해 가중치를 곱함

    return weighted_sum

# 7. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi):
    combined_index = {}
    years = [str(year) for year in range(2008, 2024)]  # 2008~2023년

    # 각 카테고리별로 종합 점수 계산
    for category in categories:
        category_score = {}

        # 연도별로 종합지표 계산
        for year in years:
            category_score[year] = calculate_weighted_score(df_csi, category, year)

        combined_index[category] = category_score

    return combined_index

# 8. 결과 출력 및 CSV 파일로 저장
combined_index = compute_combined_index_for_all_years(df_csi)
print(combined_index)

# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index)

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")

<ipython-input-7-ea550d36e6b6>:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
<ipython-input-7-ea550d36e6b6>:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


{'경제인식조사': {'2008': 2008/07    52.981818
2008/08    64.254545
2008/09    62.000000
2008/10    54.300000
2008/11    54.700000
2008/12    51.863636
dtype: float64, '2009': 2009/01     56.363636
2009/02     60.027273
2009/03     56.218182
2009/04     74.981818
2009/05     86.090909
2009/06     88.900000
2009/07     91.963636
2009/08    100.790909
2009/09    101.509091
2009/10    105.681818
2009/11    100.636364
2009/12     98.190909
dtype: float64, '2010': 2010/01    97.990909
2010/02    95.609091
2010/03    95.509091
2010/04    95.236364
2010/05    96.363636
2010/06    98.309091
2010/07    98.109091
2010/08    97.790909
2010/09    92.872727
2010/10    91.245455
2010/11    94.672727
2010/12    93.236364
dtype: float64, '2011': 2011/01    90.218182
2011/02    87.418182
2011/03    76.036364
2011/04    79.309091
2011/05    85.181818
2011/06    83.227273
2011/07    82.136364
2011/08    78.445455
2011/09    75.172727
2011/10    76.936364
2011/11    82.036364
2011/12    78.000000
dtype: float64

In [34]:
import pandas as pd
import re
# 1. CSV 파일 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_23011326.csv')

# 2. 필요한 열만 추출 (연도/월 데이터 열)
columns_to_keep = ['CSI코드', 'CSI분류코드']
colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2009, 2024) for month in range(1, 13)]
colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number
df = df[colums_to_total]

# 쉼표 및 공백 제거
df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. CSI 항목과 가중치 설정
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.2 / 0.35, '현재경기판단CSI': 0.15 / 0.35}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {
            '생활형편전망CSI': 0.15 / 0.4,
            '향후경기전망CSI': 0.1 / 0.4,
            '취업기회전망CSI': 0.1 / 0.4,
            '금리수준전망CSI': 0.05 / 0.4
        }
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI'],
        '항목별가중치': {'가계수입전망CSI': 0.1 / 0.15, '소비지출전망CSI': 0.05 / 0.15}
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.05 / 0.4, '가계저축전망CSI': 0.05 / 0.4, '현재가계부채CSI': 0.3 / 0.4}
    }
}

# 4. CSI 데이터를 피벗하여 다룬다.
df_csi = df

#print(df_csi)

# 5. 연도별 종합 점수 계산
def calculate_weighted_score(df_csi, category, year):
    weighted_sum = 0

    # 각 CSI 항목에 대한 가중치 적용
    for item, weight in categories[category]['항목별가중치'].items():
        # 연도별 월별 데이터를 선택
        monthly_columns = [col for col in df_csi.columns if col.startswith(year)]  # 연도별 월별 데이터를 선택
        df_select = df_csi[df_csi['CSI코드']==item]

        monthly_values = df_select[monthly_columns].values

        # 월별 값의 평균을 구해 가중치를 곱함
        weighted_sum += monthly_values.mean() * weight

    return weighted_sum

# 6. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi):
    combined_index = {}
    years = [str(year) for year in range(2009, 2023)]  # 2008~2023년

    # 각 연도별로 종합 점수 계산
    for year in years:
        category_scores = {}
        # 각 카테고리별 종합 점수 계산
        for category in categories:
            category_scores[category] = calculate_weighted_score(df_csi, category, year)

        combined_index[year] = category_scores

    return combined_index

# 7. 결과 출력 및 CSV 파일로 저장
combined_index = compute_combined_index_for_all_years(df_csi)


# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index).T  # Transpose로 연도별로 종합 점수가 나오도록

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")


CSV 파일로 저장 완료!


<ipython-input-34-ca81e6475c9c>:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
<ipython-input-34-ca81e6475c9c>:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


In [ ]:
import pandas as pd
import re
# 1. CSV 파일 불러오기 (예시로 'csi_data.csv'로 가정)
df = pd.read_csv('/content/drive/MyDrive/소비자동향조사(전국, 월, 2008.9~)_23011326.csv')

# 2. 필요한 열만 추출 (연도/월 데이터 열)
columns_to_keep = ['CSI코드', 'CSI분류코드']
colums_to_number = [str(year) + '/' + str(month).zfill(2) for year in range(2009, 2024) for month in range(1, 13)]
colums_to_total = ['CSI코드', 'CSI분류코드'] + colums_to_number
df = df[colums_to_total]

# 쉼표 및 공백 제거
df = df.applymap(lambda x: x.replace('·', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

# 3. CSI 항목과 가중치 설정
categories = {
    '경제인식조사': {
        '항목': ['현재생활형편CSI', '현재경기판단CSI'],
        '항목별가중치': {'현재생활형편CSI': 0.2 / 0.35, '현재경기판단CSI': 0.15 / 0.35}
    },
    '경제전망조사': {
        '항목': ['생활형편전망CSI', '향후경기전망CSI', '취업기회전망CSI', '금리수준전망CSI'],
        '항목별가중치': {
            '생활형편전망CSI': 0.15 / 0.4,
            '향후경기전망CSI': 0.1 / 0.4,
            '취업기회전망CSI': 0.1 / 0.4,
            '금리수준전망CSI': 0.05 / 0.4
        }
    },
    '소비지출전망조사': {
        '항목': ['가계수입전망CSI', '소비지출전망CSI'],
        '항목별가중치': {'가계수입전망CSI': 0.1 / 0.15, '소비지출전망CSI': 0.05 / 0.15}
    },
    '가계저축 및 부채조사': {
        '항목': ['현재가계저축CSI', '가계저축전망CSI', '현재가계부채CSI'],
        '항목별가중치': {'현재가계저축CSI': 0.05 / 0.4, '가계저축전망CSI': 0.05 / 0.4, '현재가계부채CSI': 0.3 / 0.4}
    }
}

# 4. CSI 데이터를 피벗하여 다룬다.
df_csi = df

#print(df_csi)

# 5. 연도별 종합 점수 계산
def calculate_weighted_score(df_csi, category, year):
    weighted_sum = 0

    # 각 CSI 항목에 대한 가중치 적용
    for item, weight in categories[category]['항목별가중치'].items():
        # 연도별 월별 데이터를 선택
        monthly_columns = [col for col in df_csi.columns if col.startswith(year)]  # 연도별 월별 데이터를 선택
        df_select = df_csi[df_csi['CSI코드']==item]

        monthly_values = df_select[monthly_columns].values

        # 월별 값의 평균을 구해 가중치를 곱함
        weighted_sum += monthly_values.mean() * weight

    return weighted_sum

# 6. 모든 년도별 종합지표 계산 (연도별)
def compute_combined_index_for_all_years(df_csi):
    combined_index = {}
    years = [str(year) for year in range(2009, 2023)]  # 2008~2023년

    # 각 연도별로 종합 점수 계산
    for year in years:
        category_scores = {}
        # 각 카테고리별 종합 점수 계산
        for category in categories:
            category_scores[category] = calculate_weighted_score(df_csi, category, year)

        combined_index[year] = category_scores

    return combined_index

# 7. 결과 출력 및 CSV 파일로 저장
combined_index = compute_combined_index_for_all_years(df_csi)


# DataFrame으로 변환하여 CSV로 저장
final_result = pd.DataFrame(combined_index).T  # Transpose로 연도별로 종합 점수가 나오도록

# 파일로 저장
final_result.to_csv('/content/drive/MyDrive/combined_index_by_category.csv', encoding='utf-8-sig')

print("CSV 파일로 저장 완료!")


In [3]:
import pandas as pd
import re  # 정규 표현식 모듈

# 예시 CSV 파일 경로 (실제 CSV 파일 경로를 입력하세요)
file_path = '/content/drive/MyDrive/기업경기조사(실적)_23022123.csv'

# CSV 파일에서 데이터를 읽어오기
df = pd.read_csv(file_path, encoding='utf-8')

# 불필요한 열 제거
df = df.drop(['통계표', '단위', '변환'], axis=1)

df.fillna(0, inplace=True)

# BSI코드 열에서 항목명만 추출 (숫자와 괄호 제거)
def extract_bsi_name(bsi_code):
    # 정규식: 숫자와 괄호를 제거하고, 항목명만 추출
    match = re.match(r'([^\d\(\)]+)', bsi_code)
    return match.group(1).strip() if match else bsi_code.strip()

# BSI코드 열에서 항목명만 추출하여 새로운 열로 저장
df['BSI항목'] = df['BSI코드'].apply(extract_bsi_name)

importance_weights = {
    '업황실적BSI': 0.25,
    '매출실적BSI': 0.30,
    '자금사정실적BSI': 0.35,
    '인력사정실적BSI': 0.10
}

# 날짜 열 목록 추출 (열 이름에 연도/월이 포함된 열만 추출)
date_columns = [col for col in df.columns if re.match(r'\d{4}/\d{2}', col)]

# 연도별 종합 점수 계산 함수
def weighted_average(row):
    weighted_values = 0

    # 연도별 데이터를 처리
    yearly_data = {}

    for col in date_columns:
        year = col.split('/')[0]  # '2008/01'에서 '2008'만 추출
        bsi_name = row['BSI항목']
        weight = importance_weights.get(bsi_name, 0)  # 기본 가중치는 0

        if year not in yearly_data:
            yearly_data[year] = []

        yearly_data[year].append(float(row[col]) * weight)

    # 각 연도별로 평균 계산
    for year, values in yearly_data.items():
        weighted_values += sum(values) / len(values)

    return weighted_values

# 종합 변수 계산 (모든 연도에 대해 자동으로 계산)
df['종합 변수'] = df.apply(weighted_average, axis=1)

# 결과 출력 (업종별로 종합 변수 확인)
#print(df[['업종코드', '종합 변수']])

# CSV 파일로 저장 (결과를 새로운 CSV 파일로 저장)
df.to_csv('업종별_종합_결과.csv', index=False)

# 종합 지표를 연도별로 평균 계산하여 CSV 파일로 저장할 수도 있음
df_melted = pd.melt(df, id_vars=['업종코드', 'BSI항목'], value_vars=date_columns, var_name='년도', value_name='BSI')

# 연도별 평균 종합 지수 계산
df_melted['연도'] = df_melted['년도'].apply(lambda x: x.split('/')[0])  # 연도만 추출
annual_summary = df_melted.groupby('연도')['BSI'].mean().reset_index()

# 연도별 평균 종합 지표 CSV로 저장
annual_summary.to_csv('연도별_종합_지표.csv', index=False)